In [ ]:
# default_exp web_scraping

# web scraping

> API details.

In [ ]:
#export

#Webscrapping
from time import sleep
from selenium import webdriver
import datetime

#Save data
import re
import pandas as pd

#Resources
import random
from fastcore.script import * # @Callparser
from swfd.resources import getInfo


getHtmlSfu(): Funcion que  por medio del driver de selenium hace el log en la EESSA y obtiene el código html conla información

In [ ]:
#export
def getHtmlSfu(currentdate,originaldate):

    source_code=""
    try:
        op = webdriver.ChromeOptions()
        op.add_argument('headless')
        driver = webdriver.Chrome(options=op)
        currentday = datetime.datetime.now()
        driver.get("https://esc.cbk.waw.pl/products/api.php?parameter=f10_7&start_date="+str(originaldate.year)+"%2F"+str(originaldate.month)+"%2F"+str(originaldate.day)+"+00%3A00&end_date="+str(currentdate.year)+"%2F"+str(currentdate.month)+"%2F"+str(currentdate.day)+"+00%3A00&output_type=html")
        sleep(random.uniform(8,10))

        
        user = driver.find_element_by_name("callback_0")
        password = driver.find_element_by_name("callback_1")
        button = driver.find_element_by_name("callback_2")

        credentials=getCredentials()
        
        if(credentials[0] and credentials[1]):
            user.send_keys(credentials[0])
            password.send_keys(credentials[1])
            button.click()
            sleep(random.uniform(15,10))

            elem = driver.find_element_by_xpath("//*")
            source_code = elem.get_attribute("outerHTML")
        else:
            print("user or password not found in txt")

    except: 
        e = sys.exc_info()[0]
        print("Exception: ", e)
        print("Make sure you have the internet and the selenium driver is in the sfw.")
    finally:
        driver.quit()
    
    return source_code




getCredentials(): Funcion que obtiene el user y password previamente escrito en el fichero infouser.txt para el login de la EESA

In [ ]:
#export
def getCredentials():
    credentials=[]
    user=getInfo("user")
    password=getInfo("password")
    credentials.append(user)
    credentials.append(password)
    return credentials


In [ ]:
#hide
#If txt not exist return [[], []]
getCredentials()

[[], []]

getDatosSfu(html): Función que analiza y obtiene los datos (fecha y valor de sfu) del html

In [ ]:
#export
def getListDataSfu(html):

    datalist=[]
    
    oldsfu=re.findall('<tbody>(.*?)<td>1996-02-14',str(html),re.DOTALL)
    dailysfu=re.findall('<tr>(.*?)</tr>',str(oldsfu),re.DOTALL)
    for day in dailysfu:
        if(re.findall("(.*?)00:00:00</td>",day)):
            datedata=re.findall("<td>(.*?) 00:00:00</td>",day)
            sfudata=re.findall("</td><td>(.*?)<",day)
            datalist.append([datedata[0],sfudata[0]])
    
    newsfu=re.findall('<td>1996-02-13 00:00:00</td><td>68</td></tr>(.*?)</tbody></table></div></body></html>',str(html),re.DOTALL)
    dailysfu=re.findall('<tr>(.*?)</tr>',str(newsfu),re.DOTALL)
    for day in dailysfu:
        if(re.findall("(.*?)20:00:00</td>",day)):
            datedata=re.findall("<td>(.*?) 20:00:00</td>",day)
            sfudata=re.findall("</td><td>(.*?)<",day)
            datalist.append([datedata[0],sfudata[0]])           

    return datalist[::-1]

fixDates():Función que añade las fechas faltantes de los datos

In [ ]:
#export
def fixDates(datosSfu):
    alldatasfu=[]
    #Dia actual porque el primer registro siempre sera hoy-1
    csvdaybefore=datetime.datetime.combine(datetime.datetime.today(), datetime.time.min)
    #csvdaybefore=csvdaybefore+datetime.timedelta(days=1)
    for record in datosSfu:
        realdaybefore=csvdaybefore-datetime.timedelta(days=1)
        csvdaybefore = datetime.datetime.strptime(record[0], '%Y-%m-%d')
        #Si el dia de ayer no es igual al de hoy-1 faltan datos de un dia
        if not(realdaybefore == csvdaybefore):
            while(not(realdaybefore == csvdaybefore)):
                alldatasfu.append([realdaybefore.strftime('%Y-%m-%d'),"-1"])
                realdaybefore=realdaybefore-datetime.timedelta(days=1)
                
                
        alldatasfu.append(record)
    return alldatasfu

In [ ]:
#hide
#This test was 021-05-13
fixDates([ ['2021-05-11',"-1"] , ['2021-05-09',"-1"], ['2021-05-07',"-1" ]])

[['2021-05-17', '-1'],
 ['2021-05-16', '-1'],
 ['2021-05-15', '-1'],
 ['2021-05-14', '-1'],
 ['2021-05-13', '-1'],
 ['2021-05-12', '-1'],
 ['2021-05-11', '-1'],
 ['2021-05-10', '-1'],
 ['2021-05-09', '-1'],
 ['2021-05-08', '-1'],
 ['2021-05-07', '-1']]

fixSfuValues(): Función que arregla los valores vacíos (-1) cambiandolos por la media entre los 2 valores no vacíos más cercanos

In [ ]:
#export
def fixSfuValues(datossfu):
    meanvalue=-1
    if datossfu[0][1]=='-1':
        cont=1
        while datossfu[0][1] == '-1' and cont<len(datossfu):
            if datossfu[cont][1] != '-1':
                datossfu[0][1]=datossfu[cont][1]
            cont=cont+1
    
    for i in range (1, len(datossfu)-1):
        if(datossfu[i][1]=='-1'):
            cont=i
            while datossfu[i][1] == '-1' and cont<len(datossfu):
                if datossfu[cont][1] != '-1':
                    datossfu[i][1]=str((float(datossfu[i-1][1])+float(datossfu[cont][1]))/2)
                cont=cont+1
    
    if(datossfu[len(datossfu)-1][1]=='-1'):
        datossfu[len(datossfu)-1][1]=datossfu[len(datossfu)-2][1]
        
    return datossfu

In [ ]:
#hide
testfixvalues=fixSfuValues([["date",'-1'],["date",'5'],["date",'-1'],["date",'-1'],["date",'4'],["date",'-1']])
print(testfixvalues)

[['date', '5'], ['date', '5'], ['date', '4.5'], ['date', '4.25'], ['date', '4'], ['date', '4']]


fixValues(): Función que llama a las funciones para crear las fechas faltantes con sus valores aproximados 

In [ ]:
#export
def fixValues(datossfu):
    fixdata=fixDates(datossfu)
    fixdata=fixSfuValues(fixdata)
    return fixdata
  

sfuScv(listaDatos): Función que guarda los datos(fecha y valor de sfu) en un fichero csv

In [ ]:
#export
def sfuScv(datalist):
    path=getInfo("csvdirectory")+"sfuData.csv"
    df = pd.DataFrame(datalist, columns= ['Date', 'Sfu'])
    df.to_csv(path, header = False, index = False)

In [ ]:
#hide
sfuScv([ ['2021-05-11',"-1"] , ['2021-05-09',"75"], ['2021-05-08',"55" ],['2021-05-07',"50" ],['2021-05-06',"130" ]])
path=getInfo("csvdirectory")+"sfuData.csv"
df = pd.read_csv(path,header=None)
df

,0,1
0,2021-05-11,-1
1,2021-05-09,75
2,2021-05-08,55
3,2021-05-07,50
4,2021-05-06,130


web_scraping(): Función general que obtiene los valores de la ESA, trata los datos y los guarda en un scv para posterior uso

In [ ]:
#export
def webScraping():
    datosSfu=[]
    
    currentdate = datetime.datetime.now()
    originaldate=datetime.datetime(1949, 1, 1)
    sourcecode=getHtmlSfu(currentdate,originaldate)
    pagelogin=re.findall("<title>OpenAM - (.*?)</title>",sourcecode)
    print(sourcecode)
    if(sourcecode!="" and not(pagelogin) ):
        datosSfu=getListDataSfu(sourcecode)
        datosSfu=fixValues(datosSfu)

        sfuScv(datosSfu)
        print("Sfu updated")    
    elif(pagelogin):
        if(pagelogin[0]=="Login"):
            print("Please check that your username and password are entered correctly. ")
    


getMaxValueSfu(): Saca el mayor valor de sfu historicamente

In [ ]:
#export
@call_parse
def main():
    webScraping()


In [ ]:
#main()

In [ ]:
"""for i in range (0, len(prueba2)):
    if prueba2[i][1]=="-1":
        print(variablefix[i-1][1],"   ",variablefix[i][1],"    ",variablefix[i+1][1]," el valor era",prueba2[i][1])"""

76.5     76.2      75.9  el valor era -1
81.7     81.80000000000001      81.85000000000001  el valor era -1
81.80000000000001     81.85000000000001      81.9  el valor era -1
72.1     73.5      74.9  el valor era -1
141.8     150.2      158.6  el valor era -1
134.7     133.8      132.9  el valor era -1
91.1     91.0      90.9  el valor era -1
67.7     68.4      69.1  el valor era -1
71.3     71.55      71.8  el valor era -1
84.3     85.65      87  el valor era -1
102.4     94.2      86  el valor era -1
75.8     76.85      77.9  el valor era -1
136     138.0      140  el valor era -1
90     88.5      87  el valor era -1
180     174.5      169  el valor era -1
78     78.0      78  el valor era -1
72     71.5      71  el valor era -1
73     73.5      74  el valor era -1
69     69.0      69  el valor era -1
166     162.0      158  el valor era -1
110     108.0      106  el valor era -1
146     153.5      161  el valor era -1
71     71.5      72  el valor era -1
75     75.5      76  el valo